In [26]:
import pandas as pd
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('store.csv')
df = pd.merge(df1, df2, left_on='Store', right_on='Store', how='left')

# Chcemy jeszcze wymieszać kolejność
df = df.sample(len(df))
df

/tmp/ipykernel_2980/1062901942.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('train.csv')


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
275889,294,3,2014-11-19,4983,727,1,0,0,0,a,a,3150.0,5.0,2005.0,0,NaN,NaN,NaN
165930,911,4,2015-03-05,10125,906,1,1,0,0,a,c,16490.0,NaN,NaN,0,NaN,NaN,NaN
937853,924,3,2013-03-13,3922,418,1,0,0,0,a,a,6420.0,4.0,2011.0,1,1.0,2012.0,"Jan,Apr,Jul,Oct"
760549,905,1,2013-08-19,8194,1380,1,0,0,1,a,a,90.0,6.0,2014.0,0,NaN,NaN,NaN
224507,393,7,2015-01-11,0,0,0,0,0,0,d,c,4820.0,3.0,2008.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298192,122,7,2014-10-26,2865,339,1,0,0,0,a,c,58260.0,4.0,2013.0,0,NaN,NaN,NaN
842550,396,4,2013-06-06,11895,1382,1,1,0,0,a,c,23130.0,NaN,NaN,0,NaN,NaN,NaN
834784,435,4,2013-06-13,2686,326,1,0,0,0,a,a,910.0,NaN,NaN,0,NaN,NaN,NaN
644916,117,6,2013-11-30,3168,394,1,0,0,0,a,a,3450.0,9.0,2011.0,0,NaN,NaN,NaN


In [27]:
# Główna rzecz jaką chcielibyśmy zmienić to jakoś skonwertować rzeczy, które nie są oczywiste numerycznie
# Takie rzeczy to DayOfWeek, StateHoliday
# 1 podejściem będzie one-hot-encoding
df = pd.get_dummies(df, columns=['DayOfWeek','StateHoliday', 'StoreType', "Assortment"])
df['CompetitionDistance'] = df['CompetitionDistance'].fillna(100000)
df.columns

Index(['Store', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'SchoolHoliday',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'DayOfWeek_1', 'DayOfWeek_2',
       'DayOfWeek_3', 'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6',
       'DayOfWeek_7', 'StateHoliday_0', 'StateHoliday_0', 'StateHoliday_a',
       'StateHoliday_b', 'StateHoliday_c', 'StoreType_a', 'StoreType_b',
       'StoreType_c', 'StoreType_d', 'Assortment_a', 'Assortment_b',
       'Assortment_c'],
      dtype='object')

In [28]:
# Dzielimy training na 
training_len = len(df) // 10 * 8
dev_len = len(df) // 10
test_len = len(df) - dev_len - training_len

df_train = df.iloc[:training_len]                       # 1 - 813759 train
df_dev = df.iloc[training_len:dev_len + training_len]   # 813760 - 915479 dev
df_test = df.iloc[dev_len + training_len:]              # 915480 - 1017208 test

list_inputs = list(df.columns)
to_remove = ['Date', 'Sales', 'CompetitionOpenSinceMonth', 'Customers', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'Promo2SinceWeek', 'PromoInterval']
list_inputs = [x for x in list_inputs if x not in to_remove]

X_train, Y_train = df_train[list_inputs], df_train['Sales']
X_test, Y_test = df_test[list_inputs], df_test['Sales']
X_test


,Store,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,...,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c
449111,552,0,0,0,4260.0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
154243,374,0,0,0,1150.0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
827281,737,1,1,0,100.0,1,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
386801,982,1,0,1,21930.0,0,0,0,0,1,...,0,0,0,0,0,0,1,1,0,0
831856,852,0,0,0,940.0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298192,122,1,0,0,58260.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
842550,396,1,1,0,23130.0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
834784,435,1,0,0,910.0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
644916,117,1,0,0,3450.0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [29]:
list_inputs

['Store',
 'Open',
 'Promo',
 'SchoolHoliday',
 'CompetitionDistance',
 'Promo2',
 'DayOfWeek_1',
 'DayOfWeek_2',
 'DayOfWeek_3',
 'DayOfWeek_4',
 'DayOfWeek_5',
 'DayOfWeek_6',
 'DayOfWeek_7',
 'StateHoliday_0',
 'StateHoliday_0',
 'StateHoliday_a',
 'StateHoliday_b',
 'StateHoliday_c',
 'StoreType_a',
 'StoreType_b',
 'StoreType_c',
 'StoreType_d',
 'Assortment_a',
 'Assortment_b',
 'Assortment_c']

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
model = LinearRegression()
model.fit(X_train, Y_train)

# Predict sales
Y_pred = model.predict(X_test)

# Evaluate model performance
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
print(f"RMSE: {rmse:.2f}")


RMSE: 2498.74


In [31]:
Y_pred = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(Y_train, Y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 2512.50
